In [1]:
import pygame, random, sys, os, time

from pygame.locals import *

#Declaring window height and width
WINDOW_WIDTH = 800
WINDOW_HEIGHT = 600

#Declaring text and road color
TEXT_COLOR = (255, 255, 255)
ASPHALT = (0, 0, 0)
FPS = 40

#Sprite properties
SPRITEMINSIZE = 11
SPRITEMAXSIZE = 39
SPRITEMINSPEED = 9
SPRITEMAXSPEED = 9
ADDNEWSPRITERATE = 6
PLAYERMOVERATE = 5

#Initiating score and high score
count = 3
highest_score=0

'''shutdown'''
def shutdown():
    pygame.quit()
    sys.exit()

'''Shut down and Key press'''
def initiation():
    while True:
        for event in pygame.event.get():
            if event.type == QUIT: #quit game
                shutdown()
                
            if event.type == KEYDOWN:
                if event.key == K_ESCAPE:  # quit game
                    shutdown()
                return


'''Define collision characteristics'''
def collosionReaction(rectanglePlayer, sprites):

    for b in sprites:
        if rectanglePlayer.colliderect(b['rect']):

            return True

    return False


'''Define function for text overlay'''
def textOverlay(text, font, surface, x, y):
    objectText = font.render(text, 1, TEXT_COLOR)
    rectangleText = objectText.get_rect()
    rectangleText.topleft = (x, y)
    surface.blit(objectText, rectangleText)

'''Initilizing pygame'''
pygame.init()
mainClock = pygame.time.Clock()
surfaceWindow = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption('Reckless') #Title for the game
pygame.mouse.set_visible(False) #Because mouse is not needed

#Loading sprites
header =pygame.image.load('sprites/header.png')
playerImage = pygame.image.load('sprites/car1.png')
car3 = pygame.image.load('sprites/car3.png')
car4 = pygame.image.load('sprites/car4.png')
rectanglePlayer = playerImage.get_rect()

spritePic = pygame.image.load('sprites/car2.png')

sample = [car3, car4, spritePic] #Random Generation
leftWall = pygame.image.load('sprites/left.png')
rightWall = pygame.image.load('sprites/right.png')


# Initial Game frames
font = pygame.font.SysFont(None, 42)
textOverlay('PRESS ANY KEY TO BE RECKLESS!', font, surfaceWindow, (WINDOW_WIDTH / 3) - 137, (WINDOW_HEIGHT / 3)+80) #Opening Scene
pygame.display.update() #Update Screen
initiation()

zero = 0

while (count > 0):
    sprites = [] # Initilizing sprite array
    score = 0 #Initilizing score counter
    rectanglePlayer.topleft = (WINDOW_WIDTH / 2, WINDOW_HEIGHT - 50) #player location initial
    moveLeft = moveRight = moveUp = moveDown = False # Initilizing movement variables
    rCheat = sCheat = False
    spriteCount = 0

    while True:
        score += 1

        for event in pygame.event.get():
            if event.type == QUIT:
                shutdown()

            if event.type == KEYDOWN:
                if event.key == ord('z'):
                    rCheat = True
                if event.key == ord('x'):
                    sCheat = True
                if event.key == K_LEFT or event.key == ord('a'): #key press setting for left key
                    moveRight = False
                    moveLeft = True
                if event.key == K_RIGHT or event.key == ord('d'): #key press setting for right key
                    moveLeft = False
                    moveRight = True
                if event.key == K_UP or event.key == ord('w'): #key press setting for up key
                    moveDown = False
                    moveUp = True
                if event.key == K_DOWN or event.key == ord('s'): #key press setting for down key
                    moveUp = False
                    moveDown = True

            if event.type == KEYUP:
                if event.key == ord('z'):
                    rCheat = False
                    score = 0
                if event.key == ord('x'):
                    sCheat = False
                    score = 0
                if event.key == K_ESCAPE:
                    shutdown()
                if event.key == K_LEFT or event.key == ord('a'): #key leave setting for left key
                    moveLeft = False
                if event.key == K_RIGHT or event.key == ord('d'): #key leave setting for right key
                    moveRight = False
                if event.key == K_UP or event.key == ord('w'): #key leave setting for up key
                    moveUp = False
                if event.key == K_DOWN or event.key == ord('s'): #key leave setting for down key
                    moveDown = False


        #Random sprite generation
        if not rCheat and not sCheat:
            spriteCount += 1
        if spriteCount == ADDNEWSPRITERATE:
            spriteCount = 0
            spriteSize = 30
            newBaddie = {'rect': pygame.Rect(random.randint(140, 485), 0 - spriteSize, 23, 47),
                         'speed': random.randint(SPRITEMINSPEED, SPRITEMAXSPEED),
                         'surface': pygame.transform.scale(random.choice(sample), (23, 47)),

                         }

            sprites.append(newBaddie)

            sideLeft = {'rect': pygame.Rect(0, 0, 126, 600),
                        'speed': random.randint(SPRITEMINSPEED, SPRITEMAXSPEED),
                        'surface': pygame.transform.scale(leftWall, (126, 599)),
                        }
            sprites.append(sideLeft)
            sideRight = {'rect': pygame.Rect(497, 0, 303, 600),
                         'speed': random.randint(SPRITEMINSPEED, SPRITEMAXSPEED),
                         'surface': pygame.transform.scale(rightWall, (303, 599)),
                         }
            sprites.append(sideRight)
            


        #Player movement setting
        if moveLeft and rectanglePlayer.left > 0:
            rectanglePlayer.move_ip(-1 * PLAYERMOVERATE, 0)
        if moveRight and rectanglePlayer.right < WINDOW_WIDTH:
            rectanglePlayer.move_ip(PLAYERMOVERATE, 0)
        if moveUp and rectanglePlayer.top > 0:
            rectanglePlayer.move_ip(0, -1 * PLAYERMOVERATE)
        if moveDown and rectanglePlayer.bottom < WINDOW_HEIGHT:
            rectanglePlayer.move_ip(0, PLAYERMOVERATE)

        
        #Rest of the buttons
        for b in sprites: 
            if not rCheat and not sCheat:
                b['rect'].move_ip(0, b['speed'])

            elif rCheat:
                b['rect'].move_ip(0, -5)

            elif sCheat:
                b['rect'].move_ip(0, 1)

        for b in sprites[:]:
            if b['rect'].top > WINDOW_HEIGHT:
                sprites.remove(b)

        font = pygame.font.SysFont(None, 38) #font setting
        surfaceWindow.fill(ASPHALT) #Road background color
                            
        textOverlay('Score: %s' % (score), font, surfaceWindow, 128, 1) #score tracking
        textOverlay('Top Score: %s' % (highest_score), font, surfaceWindow, 128, 21) # Top score display
        textOverlay('Rest Life: %s' % (count), font, surfaceWindow, 128, 41) # Life tracker


        surfaceWindow.blit(playerImage, rectanglePlayer)

        for b in sprites:
            surfaceWindow.blit(b['surface'], b['rect']) #sprite display


        surfaceWindow.blit(header,(0,0)) #screen display
        pygame.display.update()



        if collosionReaction(rectanglePlayer, sprites): #collision reaction
            if score > highest_score:
                highest_score = score
            break



        mainClock.tick(FPS)

    count = count - 1 #life updation
    time.sleep(1)
    font = pygame.font.SysFont(None, 52) #crash screen generation
    if (count == 0):

        textOverlay('YOU HAVE CRASHED', font, surfaceWindow, (WINDOW_WIDTH / 3)- 30, (WINDOW_HEIGHT / 3)+70)
        textOverlay('PRESS ANY KEY TO RESTART', font, surfaceWindow, (WINDOW_WIDTH /3) - 80, (WINDOW_HEIGHT / 3) + 125)
        pygame.display.update()
        time.sleep(2)
        initiation()

        count = 3



pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
